In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=c95ba513b8ea87737447a56790e3342645a001f55d94dc06cf038953cf54d9d5
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
!pip install findspark

In [ ]:

!pip install pyarrow


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import *
import time

In [ ]:
# Initialize SparkSession
spark = SparkSession.builder.appName("CSVReader").getOrCreate()

In [ ]:
# Start time
start_time = time.time()
print(f"Execution started at: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# Read the CSV data
df = spark.read.csv("/content/drive/MyDrive/CSV/yelp_review.csv", header=True, inferSchema=True)

# Display the entire dataframe (use with caution for large datasets)
df.show()

# End time
end_time = time.time()
print(f"Execution ended at: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))}")

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")


Execution started at: 2024-10-11 23:54:45
+--------------------+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|                 _id|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|66ea4ea9e59c7c5b6...|KU_O5udG6zpxOg-Vc...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|    3|     0|    0|   0|If you decide to ...|2018-07-07 22:09:11|
|66ea4ea9e59c7c5b6...|BiTunyQ73aT9WBnpR...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|    5|     1|    0|   1|I've taken a lot ...|2012-01-03 15:28:18|
|66ea4ea9e59c7c5b6...|saUsX_uimxRlCVr67...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|    3|     0|    0|   0|Family diner. Had...|2014-02-05 20:30:30|
|66ea4ea9e59c7c5b6...|AqPFMleE6RsU23_au...|_7bHUi9Uu

In [ ]:
# total records in the dataframe

total_records = df.count()
print(f"Total records in the dataframe: {total_records}")

Total records in the dataframe: 6990280


In [ ]:
# dropping the column _id

df_review = df.drop('_id')
df_review.show()

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|KU_O5udG6zpxOg-Vc...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|    3|     0|    0|   0|If you decide to ...|2018-07-07 22:09:11|
|BiTunyQ73aT9WBnpR...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|    5|     1|    0|   1|I've taken a lot ...|2012-01-03 15:28:18|
|saUsX_uimxRlCVr67...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|    3|     0|    0|   0|Family diner. Had...|2014-02-05 20:30:30|
|AqPFMleE6RsU23_au...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|    5|     1|    0|   1|Wow!  Yummy, diff...|2015-01-04 00:01:03|
|Sx8TMOWLNuJBWer-0...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|    4|     1|    0|   1|Cu

In [ ]:

# Print a valve in column "text"
for row in df_review.select("text").limit(1).collect():
  print(row.text)


If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.


In [ ]:
#  create two new columns by splitting date field to date and time,  maybe convert the original column to a timestamp after trimming  naming it date_trimmedand then, split and drop the date_trimmed

from pyspark.sql.functions import to_timestamp, split

# Assuming your date column is named 'date'
# Convert the 'date' column to timestamp after trimming and name it 'date_trimmed'
df_review = df_review.withColumn("date_trimmed", to_timestamp(trim(col("date")), "yyyy-MM-dd HH:mm:ss"))

# Split the 'date_trimmed' column into 'date' and 'time' columns
df_review = df_review.withColumn("date", split(col("date_trimmed"), " ").getItem(0))\
                   .withColumn("time", split(col("date_trimmed"), " ").getItem(1))

# Drop the 'date_trimmed' column
df_review = df_review.drop("date_trimmed")

# Show the updated DataFrame
df_review.show()


+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+----------+--------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|      date|    time|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+----------+--------+
|KU_O5udG6zpxOg-Vc...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|    3|     0|    0|   0|If you decide to ...|2018-07-07|22:09:11|
|BiTunyQ73aT9WBnpR...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|    5|     1|    0|   1|I've taken a lot ...|2012-01-03|15:28:18|
|saUsX_uimxRlCVr67...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|    3|     0|    0|   0|Family diner. Had...|2014-02-05|20:30:30|
|AqPFMleE6RsU23_au...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|    5|     1|    0|   1|Wow!  Yummy, diff...|2015-01-04|00:01:03|
|Sx8TMOWLNuJBWer-0...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|    4|     1|    0|   1|Cu

In [ ]:
# write it as a csv in /content/drive/MyDrive/ProcessedCSV named review.csv

# Write the DataFrame to a CSV file in Google Drive
df_review.write.csv("/content/drive/MyDrive/ProcessedCSV/review.csv", header=True)
